In [1]:
import pandas as pd
from pandas import *
from pandas.io.json import json_normalize

import json

pd.options.display.max_columns = 50

In [2]:
with open('../incoming/raw.json') as f:
    data = json.load(f)

In [3]:
questions = json_normalize(data, record_path=['questions'], meta=['country', 'name'])
questions['question_shared_id'] = questions.apply(lambda x: x.name % 22, axis=1)
question_params = json_normalize(data, record_path=['questions', 'params'], meta=[['questions', 'id']])
question_params.columns = ['id', 'questions.id']
question_groups = json_normalize(data, record_path='questionParams')
party_positions = json_normalize(data, record_path=['partyPositions', 'answers'],
               meta=['country', ['partyPositions', 'id'], ['partyPositions', 'name'],
                    ['partyPositions', 'party'], ['partyPositions', 'image']])

In [4]:
question_params.id = question_params.id.apply(lambda x: str(x))
question_groups.id = question_groups.id.apply(lambda x: str(x))
qpg = pd.merge(question_params, question_groups, on='id', how='left')
qpg.value = qpg.value.fillna(qpg['id'])

In [5]:
def map_to_groups(x):
    result = {}

    for k in x.params.keys():
        result[qpg[(qpg['questions.id'] == x.id) & (qpg['id'] == k)].values[0][2] \
               .lower().replace(' ', '_')] = x.params[k]
        
    return result

questions.params = questions.apply(map_to_groups, axis=1)

questions = questions.join(questions.params.apply(pd.Series))

In [6]:
full = pd.merge(party_positions, questions, left_on='question_id', right_on='id')

In [7]:
full.head()

,answer,question_id,source,source_desc,source_link,country_x,partyPositions.id,partyPositions.name,partyPositions.party,partyPositions.image,description,id,params,position,value,country_y,name,question_shared_id,liberal_society,environmental_protection,european_integration,economic_liberalisation,restrictive_financial_policy,restrictive_immigration_policy,law_and_order,xy-lr,xy-lc
0,100.0,751,Leitantrag 3 des 44. ordentlichen Bundespartei...,"""Gerade der Europäischen Union stehen die Mögl...",https://www.spoe.at/wp-content/uploads/sites/7...,AT,332,SPÖ,Social Democratic Party of Austria,http://files2.societly.com.s3-website.eu-centr...,,751,"{'liberal_society': '0', 'environmental_protec...",None,Social programmes should be maintained even at...,AT,Austria,0,0,0,0,-1,-1,0,0,-1,0
1,75.0,751,Manifest zur Wahl des Europäischen Parlaments ...,"""Die nationalen Regierungen hintertreiben die ...",https://www.1europa.at/manifest,AT,339,Initiative 1 Europe,Initiative 1 Europe,http://files2.societly.com.s3-website.eu-centr...,,751,"{'liberal_society': '0', 'environmental_protec...",None,Social programmes should be maintained even at...,AT,Austria,0,0,0,0,-1,-1,0,0,-1,0
2,100.0,751,EU-Wahlprogramm der Europapartei,"""Wir wollen ein soziales Europa mit nachhaltig...",https://vote.europeangreens.eu/manifesto,AT,344,GRÜNE,The Greens - The Green Alternative,http://files2.societly.com.s3-website.eu-centr...,,751,"{'liberal_society': '0', 'environmental_protec...",None,Social programmes should be maintained even at...,AT,Austria,0,0,0,0,-1,-1,0,0,-1,0
3,25.0,751,Grundsatzprogramm 2016,"""Die Leistungen des Staates sind klar auf schl...",https://www.neos.eu/_Resources/Persistent/b0cd...,AT,472,NEOS,The New Austria and Liberal Forum,http://files2.societly.com.s3-website.eu-centr...,,751,"{'liberal_society': '0', 'environmental_protec...",None,Social programmes should be maintained even at...,AT,Austria,0,0,0,0,-1,-1,0,0,-1,0
4,25.0,751,Regierungsprogramm 2017,"""Die Sozialhilfe (Mindestsicherung NEU) ist ei...",https://www.dieneuevolkspartei.at/download/Reg...,AT,475,ÖVP,Austrian People's Party,http://files2.societly.com.s3-website.eu-centr...,,751,"{'liberal_society': '0', 'environmental_protec...",None,Social programmes should be maintained even at...,AT,Austria,0,0,0,0,-1,-1,0,0,-1,0


In [8]:
full = full[[
    u'answer',
    u'question_id',
    u'source',
    u'source_desc',
    u'source_link',
    'country_x',
    'name',
    'partyPositions.id',
    'partyPositions.party',
    'partyPositions.name',
    'partyPositions.image',
    u'value',
    u'question_shared_id'
]]

full.columns = [
    'answer',
    'question_id',
    'motivation_src',
    'motivation',
    'motivation_src_link',
    'country_code',
    'country_name',
    'party_id',
    'party_name_full',
    'party_name_short',
    'party_image',
    'question',
    'question_shared_id'  
]

full['country_name'] = full['country_name'].replace('Netherlands', 'The Netherlands')
full.loc[full.country_code == 'WA', 'question_id'] = full.loc[full.country_code == 'WA', 'question_id'].apply(lambda x: x - 22)
full.country_name.replace('Belgium - Walloon', 'Belgium', inplace=True)
full.country_name.replace('Belgium - Flanders', 'Belgium', inplace=True)
full.country_code.replace('WA', 'BE', inplace=True)
full.country_code.replace('VL', 'BE', inplace=True)

In [11]:
for c in full.country_code.unique():
    full[full.country_code == c].to_csv('../public/data/%s.csv' % c, encoding='utf-8', index=False)

In [64]:
full.to_csv('../public/data/data.csv', encoding='utf-8', index=False)